# **Test RF with ObliqueTree from KDercksen**

https://github.com/KDercksen/pyblique

In [1]:
FOLDERS= ['/content/drive/MyDrive/',
          '/content/drive/MyDrive/midas',
          '/content/drive/MyDrive/Testes e Experimentos/Testes/lib.external',
          '/content/drive/MyDrive/Testes e Experimentos/Testes/lib.external/KDercksen',
          '/content/drive/MyDrive/Testes e Experimentos/Testes/lib.external/KDercksen/pyblique']
import sys
for f in FOLDERS:
  sys.path.append(f)

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

from midas.v4.ensemble import RFClassifier

In [2]:
df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

## Testa as árvores obliquas individualmente

In [ ]:
from KDercksen.pyblique.pyblique import *
from KDercksen.pyblique.impurity import *

data_train = np.zeros((X_train.shape[0],X_train.shape[1]+1))
data_train[:, :-1] = X_train.to_numpy(copy=True)
data_train[:,-1] = y_train.to_numpy(copy=True)

# 
co = ObliqueClassifier()

co.fit(data_train)

y_pred = co.predict(X_test.to_numpy(copy=True))

print('Acc: ', accuracy_score(y_test, y_pred))

## Testa com a implementação do RF

In [ ]:
from KDercksen.pyblique.pyblique import *
from KDercksen.pyblique.impurity import *

# Continuous Optimization of Oblique Splits (CO2)

cfg_base_estimator = {}

rf = RFClassifier(base_estimator=ObliqueClassifier,
                  n_estimators=10,
                  params_estimators=cfg_base_estimator,
                  enable_logger=True)

rf._setup_logger(log_file='/content/drive/MyDrive/Testes e Experimentos/Testes/rf.kdoc.debuging.log')

rf = rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print('Acc with oblique tree', accuracy_score(y_test, y_pred))